In [1]:
import pandas as pd
import os 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from phik.report import plot_correlation_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans
import numpy as np

In [2]:
ekg_data = pd.read_csv(f"/Users/mariasemonaeva/АНАЛИТИК ДАННЫХ/important/Хакатон Кардио/ECG_7743.txt", delimiter=' ',
                 names=['I', 'II', 'V5', 'III', 'aVL', 'aVR', 'aVF'], index_col=False)
df  = pd.read_excel(f"/Users/mariasemonaeva/АНАЛИТИК ДАННЫХ/important/Хакатон Кардио/dataset_7743.xlsx")

In [3]:
df

,"время, мсек",координаты,Unnamed: 2,тип,код,арритмия,начало,31560000,мсек
0,NaN,мсек,строки,комплекса,аритмии,NaN,частота,266.0,Гц
1,31560677.0,677,181,S,NaN,,NaN,NaN,NaN
2,31561432.0,1432,381,S,NaN,,NaN,NaN,NaN
3,31562162.0,2162,576,S,NaN,,NaN,NaN,NaN
4,31562921.0,2921,777,S,NaN,,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
96875,118187248.0,86627248,23042848,S,NaN,,NaN,NaN,NaN
96876,118188038.0,86628038,23043059,S,NaN,,NaN,NaN,NaN
96877,118188850.0,86628850,23043275,S,NaN,,NaN,NaN,NaN
96878,118189658.0,86629658,23043490,S,NaN,,NaN,NaN,NaN


In [4]:
df = df.rename(columns={'время, мсек': 'время_мсек',
                'координаты': 'координаты_мсек', 
                'Unnamed: 2': 'координаты_строки', 
                'тип':'тип_комплекса',
                'код':'код_аритмии'})

In [5]:
df = df.drop(0, axis=0)
df = df.drop(['начало', 31560000, 'мсек'], axis=1)
df

,время_мсек,координаты_мсек,координаты_строки,тип_комплекса,код_аритмии,арритмия
1,31560677.0,677,181,S,NaN,
2,31561432.0,1432,381,S,NaN,
3,31562162.0,2162,576,S,NaN,
4,31562921.0,2921,777,S,NaN,
5,31563669.0,3669,976,S,NaN,
...,...,...,...,...,...,...
96875,118187248.0,86627248,23042848,S,NaN,
96876,118188038.0,86628038,23043059,S,NaN,
96877,118188850.0,86628850,23043275,S,NaN,
96878,118189658.0,86629658,23043490,S,NaN,


In [6]:
ekg_data = ekg_data.merge(df, how='left', left_index=True,  right_on='координаты_строки').reset_index()
#ekg_data_1 = ekg_data_1.drop(['index', 'время_мсек','координаты_мсек','координаты_строки'], axis=1)
#ekg_data_1 = ekg_data_1.drop(['index', 'время_мсек','координаты_мсек','координаты_строки'], axis=1)

In [7]:
ekg_data.head()

,index,I,II,V5,III,aVL,aVR,aVF,время_мсек,координаты_мсек,координаты_строки,тип_комплекса,код_аритмии,арритмия
0,NaN,1,-3,-5,-4,3,1,-4,NaN,NaN,0,NaN,NaN,NaN
1,NaN,1,-2,-5,-3,2,1,-3,NaN,NaN,1,NaN,NaN,NaN
2,NaN,1,-1,-5,-2,2,0,-2,NaN,NaN,2,NaN,NaN,NaN
3,NaN,1,0,-5,-1,1,-1,-1,NaN,NaN,3,NaN,NaN,NaN
4,NaN,1,0,-5,-1,1,-1,-1,NaN,NaN,4,NaN,NaN,NaN


In [8]:
ekg_data.код_аритмии.value_counts()

ANZA      13188
ANVES1     1555
ANVEDM      154
ANPU         62
ANVRTM        9
ANSES         8
ANVEDP        6
ANVRTP        3
ANVES3        1
Name: код_аритмии, dtype: int64

In [9]:
#ffill после значения проставить следуюшие за ним пустые строки этим значением, bfill - предыдущие. limit - количество строк которые максимально заменяются 
ekg_data = ekg_data.fillna(method='ffill', limit=100).fillna(method='bfill', limit=100)

In [10]:
ekg_data_viz = ekg_data.copy()
ekg_data_viz['код_аритмии'] = ekg_data_viz['код_аритмии'].fillna('normal')

In [11]:
ekg_data_viz.head(3)

,index,I,II,V5,III,aVL,aVR,aVF,время_мсек,координаты_мсек,координаты_строки,тип_комплекса,код_аритмии,арритмия
0,NaN,1,-3,-5,-4,3,1,-4,NaN,NaN,0,NaN,normal,NaN
1,NaN,1,-2,-5,-3,2,1,-3,NaN,NaN,1,NaN,normal,NaN
2,NaN,1,-1,-5,-2,2,0,-2,NaN,NaN,2,NaN,normal,NaN


In [12]:
ekg_data_viz['арритмия'].unique()

array([nan, ' ', 'бигеминия',
       'желудочковая одиночная экстрасистола 1-го типа', 'пауза',
       'желудочковая парная мономорфная экстрасистола',
       'наджелудочковая одиночная экстрасистола',
       'желудочковая одиночная экстрасистола 3-го типа',
       'желудочковая полиморфная тахикардия',
       'желудочковая мономорфная тахикардия',
       'желудочковая парная полиморфная экстрасистола'], dtype=object)

In [13]:
ekg_data_viz['код_аритмии'].unique()

array(['normal', 'ANZA', 'ANVES1', 'ANPU', 'ANVEDM', 'ANSES', 'ANVES3',
       'ANVRTP', 'ANVRTM', 'ANVEDP'], dtype=object)

In [14]:
ekg_data_viz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23043790 entries, 0 to 23043789
Data columns (total 14 columns):
 #   Column             Dtype  
---  ------             -----  
 0   index              float64
 1   I                  int64  
 2   II                 int64  
 3   V5                 int64  
 4   III                int64  
 5   aVL                int64  
 6   aVR                int64  
 7   aVF                int64  
 8   время_мсек         float64
 9   координаты_мсек    float64
 10  координаты_строки  int64  
 11  тип_комплекса      object 
 12  код_аритмии        object 
 13  арритмия           object 
dtypes: float64(3), int64(8), object(3)
memory usage: 2.4+ GB


In [15]:
dt = ekg_data_viz.loc[(ekg_data_viz['код_аритмии'] == 'ANZA') | (ekg_data_viz['код_аритмии'] == 'normal')]
dt.head()

,index,I,II,V5,III,aVL,aVR,aVF,время_мсек,координаты_мсек,координаты_строки,тип_комплекса,код_аритмии,арритмия
0,NaN,1,-3,-5,-4,3,1,-4,NaN,NaN,0,NaN,normal,NaN
1,NaN,1,-2,-5,-3,2,1,-3,NaN,NaN,1,NaN,normal,NaN
2,NaN,1,-1,-5,-2,2,0,-2,NaN,NaN,2,NaN,normal,NaN
3,NaN,1,0,-5,-1,1,-1,-1,NaN,NaN,3,NaN,normal,NaN
4,NaN,1,0,-5,-1,1,-1,-1,NaN,NaN,4,NaN,normal,NaN


In [16]:
dt['код_аритмии'].unique()

array(['normal', 'ANZA'], dtype=object)

In [17]:
dt = dt.drop(['арритмия', 'index', 'время_мсек', 'координаты_мсек', 'координаты_строки', 'тип_комплекса'], axis=1)

In [18]:
dt

,I,II,V5,III,aVL,aVR,aVF,код_аритмии
0,1,-3,-5,-4,3,1,-4,normal
1,1,-2,-5,-3,2,1,-3,normal
2,1,-1,-5,-2,2,0,-2,normal
3,1,0,-5,-1,1,-1,-1,normal
4,1,0,-5,-1,1,-1,-1,normal
...,...,...,...,...,...,...,...,...
23043785,10,43,-6,33,-13,-31,44,normal
23043786,-2,6,11,8,-6,-2,8,normal
23043787,-14,-31,28,-17,2,26,-28,normal
23043788,-26,-68,44,-42,9,54,-64,normal


In [19]:
df_c = pd.get_dummies(dt['код_аритмии'])
df_c.head(3)

,ANZA,normal
0,0,1
1,0,1
2,0,1


In [20]:
dtt = dt
dtt['ANZA'] = df_c['ANZA']
dtt = dt.drop(['код_аритмии'], axis=1)
dtt.head()

,I,II,V5,III,aVL,aVR,aVF,ANZA
0,1,-3,-5,-4,3,1,-4,0
1,1,-2,-5,-3,2,1,-3,0
2,1,-1,-5,-2,2,0,-2,0
3,1,0,-5,-1,1,-1,-1,0
4,1,0,-5,-1,1,-1,-1,0


In [21]:
dtt = dtt.rename(columns={'ANZA': 'бигеминия'})
dtt.head(2)

,I,II,V5,III,aVL,aVR,aVF,бигеминия
0,1,-3,-5,-4,3,1,-4,0
1,1,-2,-5,-3,2,1,-3,0


In [22]:
#Разделим наши данные на признаки (матрица X) и целевую переменную (y)
X_train = dtt.drop(columns=['бигеминия'], axis=1)
y_train = dtt['бигеминия']

#Разобьем данные на обучающую и валидационную выборку
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#Обучим модель на train-выборке логистической регрессией
# алгоритм для нашей модели
#model_lr = LogisticRegression(solver='liblinear', random_state=0)

# обучим модель
#model_lr.fit(X_train, y_train)

# воспользуемся уже обученной моделью, чтобы сделать прогнозы
# прогноз класса после обучения такой модели
#y_pred_lr = model_lr.predict(X_test)
#  вероятность принадлежности объекта к первому или второму классу
#y_probas_lr = model_lr.predict_proba(X_test)[:,1]

# вывoд значения predictions и probabilities на экран
#print(y_pred)
#print(y_probas)

Тут при объявлении модели мы задали n_estimators — число деревьев, на основании которых будем строить лес. При этом другие параметры дерева — например, глубину дерева max_depth, размер подвыборки признаков max_features, минимальное количество объектов в узле min_samples_leaf — мы не задавали и оставили их по умолчанию. На практике вы можете экспериментировать с этими значениями и смотреть, как это влияет на результат.

In [23]:
# стандартизируем данные
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)

y_train = np.array(y_train)

#Обучим модель на train-выборке случайным лесом
rfc_model = RandomForestClassifier(n_estimators=30)

# обучим модель
rfc_model.fit(X_train_sc, y_train)

RandomForestClassifier(n_estimators=30)

In [24]:
ekg_data1 = pd.read_csv(f"/Users/mariasemonaeva/АНАЛИТИК ДАННЫХ/important/Хакатон Кардио/ECG_10156.txt", delimiter=' ',
                 names=['I', 'II', 'V5', 'III', 'aVL', 'aVR', 'aVF'], index_col=False)
df1  = pd.read_excel(f"/Users/mariasemonaeva/АНАЛИТИК ДАННЫХ/important/Хакатон Кардио/dataset_10156.xlsx")

In [25]:
df1

,"время, мсек",координаты,Unnamed: 2,тип,код,арритмия,начало,40620000,мсек
0,NaN,мсек,строки,NaN,NaN,NaN,частота,266.0,Гц
1,40620417.0,417,111,Z,NaN,,NaN,NaN,NaN
2,40620805.0,805,215,Z,NaN,,NaN,NaN,NaN
3,40621759.0,1759,468,Z,NaN,,NaN,NaN,NaN
4,40622169.0,2169,577,Z,NaN,,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
70156,110489868.0,69869868,18585385,S,NaN,,NaN,NaN,NaN
70157,110490620.0,69870620,18585585,S,NaN,,NaN,NaN,NaN
70158,110491398.0,69871398,18585792,S,NaN,,NaN,NaN,NaN
70159,110491944.0,69871944,18585938,V1,ANVES1,желудочковая одиночная экстрасистола 1-го типа,NaN,NaN,NaN


In [26]:
df1 = df1.rename(columns={'время, мсек': 'время_мсек',
                'координаты': 'координаты_мсек', 
                'Unnamed: 2': 'координаты_строки', 
                'тип':'тип_комплекса',
                'код':'код_аритмии'})
df1 = df1.drop(0, axis=0)
df1 = df1.drop(['начало', 40620000, 'мсек'], axis=1)
df1

,время_мсек,координаты_мсек,координаты_строки,тип_комплекса,код_аритмии,арритмия
1,40620417.0,417,111,Z,NaN,
2,40620805.0,805,215,Z,NaN,
3,40621759.0,1759,468,Z,NaN,
4,40622169.0,2169,577,Z,NaN,
5,40623628.0,3628,966,S,NaN,
...,...,...,...,...,...,...
70156,110489868.0,69869868,18585385,S,NaN,
70157,110490620.0,69870620,18585585,S,NaN,
70158,110491398.0,69871398,18585792,S,NaN,
70159,110491944.0,69871944,18585938,V1,ANVES1,желудочковая одиночная экстрасистола 1-го типа


In [27]:
ekg_data1 = ekg_data1.merge(df1, how='left', left_index=True,  right_on='координаты_строки').reset_index()
#ffill после значения проставить следуюшие за ним пустые строки этим значением, bfill - предыдущие. limit - количество строк которые максимально заменяются 
ekg_data1 = ekg_data1.fillna(method='ffill', limit=100).fillna(method='bfill', limit=100)
ekg_data_viz1 = ekg_data1.copy()
ekg_data_viz1['код_аритмии'] = ekg_data_viz1['код_аритмии'].fillna('normal')
ekg_data_viz1.head(3)

,index,I,II,V5,III,aVL,aVR,aVF,время_мсек,координаты_мсек,координаты_строки,тип_комплекса,код_аритмии,арритмия
0,NaN,22,22,10,0,13,-25,13,NaN,NaN,0,NaN,normal,NaN
1,NaN,18,19,10,1,10,-21,12,NaN,NaN,1,NaN,normal,NaN
2,NaN,15,16,9,1,8,-18,10,NaN,NaN,2,NaN,normal,NaN


In [28]:
dt1 = ekg_data_viz1.loc[(ekg_data_viz1['код_аритмии'] == 'ANZA') | (ekg_data_viz1['код_аритмии'] == 'normal')]
dt1 = dt1.drop(['арритмия', 'index', 'время_мсек', 'координаты_мсек', 'координаты_строки', 'тип_комплекса'], axis=1)
dt1.head()

,I,II,V5,III,aVL,aVR,aVF,код_аритмии
0,22,22,10,0,13,-25,13,normal
1,18,19,10,1,10,-21,12,normal
2,15,16,9,1,8,-18,10,normal
3,14,13,8,-1,9,-16,7,normal
4,13,10,7,-3,9,-13,4,normal


In [29]:
df_c = pd.get_dummies(dt1['код_аритмии'])
df_c.head(3)

,ANZA,normal
0,0,1
1,0,1
2,0,1


In [30]:
dtt1 = dt1
dtt1['ANZA'] = df_c['ANZA']
dtt1 = dt1.drop(['код_аритмии'], axis=1)
dtt1.head()

,I,II,V5,III,aVL,aVR,aVF,ANZA
0,22,22,10,0,13,-25,13,0
1,18,19,10,1,10,-21,12,0
2,15,16,9,1,8,-18,10,0
3,14,13,8,-1,9,-16,7,0
4,13,10,7,-3,9,-13,4,0


In [31]:
dtt1 = dtt1.rename(columns={'ANZA': 'бигеминия'})
dtt1.head(2)

,I,II,V5,III,aVL,aVR,aVF,бигеминия
0,22,22,10,0,13,-25,13,0
1,18,19,10,1,10,-21,12,0


In [32]:
#Разделим наши данные на признаки (матрица X) и целевую переменную (y)
X_test = dtt1.drop(columns=['бигеминия'], axis=1)
y_test = dtt1['бигеминия']

In [33]:
# стандартизируем данные
X_test_sc = scaler.transform(X_test)

# воспользуемся уже обученной моделью, чтобы сделать прогнозы
y_pred_rfc = rfc_model.predict(X_test_sc)

#display(rfc_model.score(X_train_scaled, y_train))

In [34]:
#Оценим метрики accuracy, precision и recall для обеих моделей на валидационной выборке
print('Метрики классификаций для модели "случайный лес"')
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_pred_rfc)))
print('Precision: {:.2f}'.format(precision_score(y_test, y_pred_rfc)))
print('Recall: {:.2f}'.format(recall_score(y_test, y_pred_rfc)))
print('F1: {:.2f}'.format(f1_score(y_test, y_pred_rfc)))

Метрики классификаций для модели "случайный лес"
Accuracy: 0.78
Precision: 0.46
Recall: 0.08
F1: 0.14
